In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120)(X)
    X = Activation("relu")(X)
    
    X = Dense(84)(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
momentum_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
from keras.optimizers import SGD
beta = 0.9
optimizer = SGD(lr=0.001, momentum=beta)
momentum_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [8]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('momentum_training.log')

In [9]:
history = momentum_model.fit(X_train, Y_train, validation_data=[X_val, Y_val], batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 63s 1ms/step - loss: 1.6590 - acc: 0.4877 - val_loss: 0.4654 - val_acc: 0.8695
Epoch 2/20
50000/50000 [==============================] - 76s 2ms/step - loss: 0.3747 - acc: 0.8881 - val_loss: 0.2666 - val_acc: 0.9202
Epoch 3/20
50000/50000 [==============================] - 79s 2ms/step - loss: 0.2596 - acc: 0.9215 - val_loss: 0.2065 - val_acc: 0.9399
Epoch 4/20
50000/50000 [==============================] - 74s 1ms/step - loss: 0.2116 - acc: 0.9349 - val_loss: 0.2055 - val_acc: 0.9345
Epoch 5/20
50000/50000 [==============================] - 82s 2ms/step - loss: 0.1793 - acc: 0.9450 - val_loss: 0.1498 - val_acc: 0.9549
Epoch 6/20
50000/50000 [==============================] - 60s 1ms/step - loss: 0.1557 - acc: 0.9524 - val_loss: 0.1387 - val_acc: 0.9598
Epoch 7/20
50000/50000 [==============================] - 93s 2ms/step - loss: 0

In [10]:
(loss, accuracy) = momentum_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 7s 659us/step
accuracy: 97.85%


In [11]:
import pandas as pd
df = pd.read_csv('momentum_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.48768  1.658996   0.8695  0.465377
1       1  0.88808  0.374671   0.9202  0.266605
2       2  0.92152  0.259601   0.9399  0.206540
3       3  0.93488  0.211610   0.9345  0.205460
4       4  0.94500  0.179316   0.9549  0.149838
5       5  0.95240  0.155736   0.9598  0.138709
6       6  0.95788  0.139051   0.9623  0.126763
7       7  0.96200  0.125983   0.9663  0.113319
8       8  0.96622  0.111770   0.9655  0.115015
9       9  0.96820  0.105242   0.9713  0.100400
10     10  0.97036  0.097554   0.9683  0.105987
11     11  0.97196  0.090170   0.9739  0.089576
12     12  0.97438  0.083455   0.9751  0.086578
13     13  0.97486  0.080143   0.9771  0.082123
14     14  0.97746  0.074487   0.9755  0.081781
15     15  0.97846  0.068910   0.9785  0.074185
16     16  0.97880  0.067271   0.9764  0.077964
17     17  0.98092  0.063544   0.9800  0.069096
18     18  0.98116  0.061624   0.9787  0.071171
19     19  0.98118  0.059349   0.9780  0